In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

plt.style.use('seaborn-whitegrid')

In [2]:

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

In [3]:

!apt install fonts-nanum

import matplotlib as mpl
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [4]:
# !pip install catboost
from catboost import CatBoostRegressor, Pool

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=d464272e41f0f3a0c3c23fc8c3c8df9459cb7f0b2f06f7925460298e599a7e0b
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f9\8d\8d\f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [5]:
# !pip install shap
import shap

In [6]:
data = pd.read_csv('0924_ver.csv', encoding = 'utf-8', index_col = 0).iloc[:,1:]

In [7]:
data

,노출(분),상품명,상품군,판매단가,취급액,월,일,시,소비층,요일,휴일,프라임,파트,브랜드
0,20.0,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,1,6,1,Tuesday,2,4,First,테이트
1,20.0,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,1,6,1,Tuesday,2,4,First,테이트
2,20.0,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,1,6,1,Tuesday,2,4,Middle,테이트
3,20.0,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,1,6,1,Tuesday,2,4,Middle,테이트
4,20.0,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,1,6,1,Tuesday,2,4,Later,테이트
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35370,20.0,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,12,31,23,2,Tuesday,0,4,First,쿠첸
35371,20.0,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸
35372,20.0,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸
35373,20.0,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸


In [8]:
data['취급액_로그'] = np.log(data.취급액)
data

,노출(분),상품명,상품군,판매단가,취급액,월,일,시,소비층,요일,휴일,프라임,파트,브랜드,취급액_로그
0,20.0,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,1,6,1,Tuesday,2,4,First,테이트,14.556972
1,20.0,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,1,6,1,Tuesday,2,4,First,테이트,15.290502
2,20.0,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,1,6,1,Tuesday,2,4,Middle,테이트,14.997851
3,20.0,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,1,6,1,Tuesday,2,4,Middle,테이트,15.754971
4,20.0,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,1,6,1,Tuesday,2,4,Later,테이트,15.713430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35370,20.0,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,12,31,23,2,Tuesday,0,4,First,쿠첸,14.324735
35371,20.0,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸,16.029155
35372,20.0,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸,16.542186
35373,20.0,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸,14.660520


In [9]:
import re

def max_words(goods_kind):
    words = ' '.join(list(data[data.상품군 == goods_kind].상품명.values))

    words = re.split(r'[+ ]', words)
    words = list(filter(lambda x: x != '', words))
    word_count = {}
    for word in set(words):
        count = words.count(word)
        word_count.update({word:count})
        
    return word_count

In [10]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [11]:
def train_df(group_name, max_features):
    group_df = data[data.상품군 == group_name]

    # Apply CountVectorizer with preprocessed 상품명
    words = ' '.join(list(group_df.상품명.dropna().values))
    words = re.split(r'[()+ ]', words)
    words = list(filter(lambda x: x != '', words))

    vect = CountVectorizer(max_features = max_features)
    vect.fit(words)

    # merge base dataframe with word-embedding dataframe
    df = pd.DataFrame(np.stack(group_df.상품명.map(lambda x: vect.transform(x.split()).toarray().sum(axis=0)).values),
                      columns=vect.get_feature_names(),
                      index = group_df.index)
    df = pd.concat([group_df, df], axis=1)
    df = df.drop(columns = ['상품명','상품군'])

    return df

In [13]:
def plot_result(test_dict, predict_dict, scores_dict):
    fig,  ax = plt.subplots(2,6, figsize = (15, 5))
    for i in range(12):
        x = i//6
        y = i%6
        ax[x, y].plot(test_dict[i+1], predict_dict[i+1], linestyle = '', marker = 'o')
        ax[x, y].plot(test_dict[i+1], test_dict[i+1], color = 'r')
        ax[x, y].set_title(f'{i+1}: MAPE {round(scores_dict[i+1],2)}')

In [14]:
df = train_df('주방', 39)

In [15]:
cv_scores, cv_models, cv_test, cv_predict = LGBM_train(df)

NameError: name 'LGBM_train' is not defined

In [20]:
data['브랜드'].fillna('food')

0        테이트
1        테이트
2        테이트
3        테이트
4        테이트
        ... 
35370     쿠첸
35371     쿠첸
35372     쿠첸
35373     쿠첸
35374     쿠첸
Name: 브랜드, Length: 35375, dtype: object

In [24]:
data

,노출(분),상품명,상품군,판매단가,취급액,월,일,시,소비층,요일,휴일,프라임,파트,브랜드,취급액_로그
0,20.0,테이트 남성 셀린니트3종,의류,39900,2099000.0,1,1,6,1,Tuesday,2,4,First,테이트,14.556972
1,20.0,테이트 여성 셀린니트3종,의류,39900,4371000.0,1,1,6,1,Tuesday,2,4,First,테이트,15.290502
2,20.0,테이트 남성 셀린니트3종,의류,39900,3262000.0,1,1,6,1,Tuesday,2,4,Middle,테이트,14.997851
3,20.0,테이트 여성 셀린니트3종,의류,39900,6955000.0,1,1,6,1,Tuesday,2,4,Middle,테이트,15.754971
4,20.0,테이트 남성 셀린니트3종,의류,39900,6672000.0,1,1,6,1,Tuesday,2,4,Later,테이트,15.713430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35370,20.0,일시불쿠첸압력밥솥 6인용,주방,148000,1664000.0,12,31,23,2,Tuesday,0,4,First,쿠첸,14.324735
35371,20.0,무이자쿠첸압력밥솥 10인용,주방,178000,9149000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸,16.029155
35372,20.0,일시불쿠첸압력밥솥 10인용,주방,168000,15282000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸,16.542186
35373,20.0,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0,12,31,23,2,Tuesday,0,4,Later,쿠첸,14.660520


In [16]:
model = CatBoostRegressor(iterations = 2000,
                          loss_function="RMSE",
                          eval_metric="MAPE",
                           task_type="GPU",
                          learning_rate=0.1,
                          early_stopping_rounds=500,
                          random_seed = 123457,
                          )

In [26]:
month = list(set(df.월))

raw_x = data.drop(columns=['취급액_로그', '취급액'])
raw_y = data[['취급액_로그','월']]

cv_test = {}
cv_predict = {}
cv_scores = {}
cv_models = {}
cv_test_dataset = {}

for m in month:

    # try:

    test_x = raw_x[raw_x.월 == m].drop(columns = ['월'])
    test_y = raw_y[raw_y.월 == m].취급액_로그
    train_x = raw_x.drop(index = test_x.index, columns = ['월'])
    train_y = raw_y.drop(index = test_y.index).취급액_로그

    # test_x = mmscaler.fit_transform(test_x)
    # train_x = mmscaler.fit_transform(train_x)

    train_y = train_y.to_numpy().reshape(-1)
    test_y = test_y.to_numpy().reshape(-1)

    cat_features_train = np.where((train_x.dtypes != np.float)&(train_x.dtypes != np.int))[0]
    cat_features_test = np.where((test_x.dtypes != np.float)&(test_x.dtypes != np.int))[0]
    
    train_dataset = Pool(train_x, label = train_y, cat_features=cat_features_train)
    test_dataset = Pool(test_x, label = test_y, cat_features=cat_features_test)
    CBR = model.fit(train_dataset,  
                    eval_set = test_dataset,
                    use_best_model = True,      
                    verbose_eval = 500,
                    plot = False)
    predict_test = model.predict(test_x)

    test_y = np.exp(test_y)
    predict_test = np.exp(predict_test)

    cv_test.update({m:test_y})
    cv_predict.update({m: predict_test})

    mape = mean_absolute_percentage_error(test_y, predict_test)
    print(f'{m}월 MAPE:', mape)

    cv_scores.update({m:mape})
    cv_models.update({m:model})
    cv_test_dataset.update({m:test_dataset})

    print()

CatBoostError: Invalid type for cat_feature[non-default value idx=46,feature_idx=11]=nan : cat_features must be integer or string, real number values and NaN values should be converted to string.